In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, Normalizer, StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
import seaborn as sns

In [3]:
def get_high_cor(data, threshold = 0.2):
    cor = data.corr().stack().reset_index().drop_duplicates()
    cor.columns = ['FEATURE_1', 'FEATURE_2', 'CORRELATION']
    high_cor = cor[((cor['FEATURE_1'] != cor['FEATURE_2'] ) & ((cor['CORRELATION'] >= threshold) | (cor['CORRELATION'] <= threshold * -1)))]
    return high_cor

In [4]:
data_df = pd.read_csv('Data\\recs2009_public.csv')
data_df.dropna(axis=0,inplace=True)
data_df.drop(columns=["DOEID"],inplace=True)
data_df.reset_index(inplace=True, drop=True)

cols = data_df.columns.tolist()

In [5]:
for i in range(len(data_df.index)):
    for col in cols:
        if col != "METROMICRO" and col != "UR":
            if data_df.at[i, col] < 0:
                data_df.at[i, col] = 0

        elif col == "UR":
            if data_df.at[i, col] == "U":
                data_df.at[i, col] = 0
            else:
                data_df.at[i, col] = 1
        else:
            if data_df.at[i, col] == "METRO":
                data_df.at[i, col] = 0
            if data_df.at[i, col] == "MICRO":
                data_df.at[i, col] = 1
            if data_df.at[i, col] == "NONE":
                data_df.at[i, col] = 2
            

In [6]:
data_df

,REGIONC,DIVISION,REPORTABLE_DOMAIN,TYPEHUQ,NWEIGHT,HDD65,CDD65,HDD30YR,CDD30YR,Climate_Region_Pub,...,SCALEEL,KAVALNG,PERIODNG,SCALENG,PERIODLP,SCALELP,PERIODFO,SCALEFO,PERIODKR,SCALEKER
0,2,4,12,2,2471.679705,4742,1080,4953,1271,4,...,0,0,0,0,0,0,0,0,0,0
1,4,10,26,2,8599.172010,2662,199,2688,143,5,...,0,1,1,0,0,0,0,0,0,0
2,1,1,1,5,8969.915921,6233,505,5741,829,1,...,0,3,5,3,0,0,0,0,0,0
3,2,3,7,2,18003.639600,6034,672,5781,868,1,...,3,3,5,3,0,0,0,0,0,0
4,1,1,1,3,5999.605242,5388,702,5313,797,1,...,0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12076,2,3,9,2,10219.802880,7671,332,7784,451,1,...,0,3,5,3,0,0,0,0,0,0
12077,3,7,21,4,8689.074445,2085,2844,2520,2286,2,...,0,1,1,0,0,0,0,0,0,0
12078,2,4,10,2,4697.213624,7935,472,8074,600,1,...,0,1,1,0,0,0,0,0,0,0
12079,4,8,23,2,13155.678790,5834,770,5989,957,1,...,0,1,1,0,0,0,0,0,0,0


In [7]:
# BTUEL is KWH just in another unit of measurement and KWH*** are a separation of KWH on specific tasks such as airconditioning.

KWH_cheat = [i for i in data_df.columns if(("KWH" in i and len(i)>=4 or "BTUEL" in i))]
print(KWH_cheat)

data_df.drop(columns = KWH_cheat, inplace = True)

['KWHSPH', 'KWHCOL', 'KWHWTH', 'KWHRFG', 'KWHOTH', 'BTUEL', 'BTUELSPH', 'BTUELCOL', 'BTUELWTH', 'BTUELRFG', 'BTUELOTH']


In [8]:
high_cor = get_high_cor(data_df)
cor = high_cor[((high_cor['FEATURE_1'] == 'KWH'))]


In [9]:
cor

,FEATURE_1,FEATURE_2,CORRELATION
709855,KWH,TYPEHUQ,-0.352917
709858,KWH,CDD65,0.245610
709860,KWH,CDD30YR,0.256357
709862,KWH,AIA_Zone,0.200681
709863,KWH,KOWNRENT,-0.256495
...,...,...,...
710721,KWH,TOTALDOLWTH,0.437861
710722,KWH,TOTALDOLRFG,0.433450
710723,KWH,TOTALDOLOTH,0.655677
710727,KWH,KAVALNG,-0.275771


In [10]:
corr_list = []

for elem in cor['FEATURE_2']:
    corr_list.append(elem)

high_cor_df = data_df[data_df.columns.intersection(corr_list)]

high_cor_df = pd.concat((high_cor_df, data_df['KWH']),axis = 1)

print(high_cor_df)
# high_cor_df.to_csv("./Data/high_cor_df.csv", index=False)

       TYPEHUQ  CDD65  CDD30YR  AIA_Zone  KOWNRENT  YEARMADE  YEARMADERANGE  \
0            2   1080     1271         3         1      2004              7   
1            2    199      143         4         2      1998              6   
2            5    505      829         2         2      1965              3   
3            2    672      868         2         1      1985              5   
4            3    702      797         3         1      1983              5   
...        ...    ...      ...       ...       ...       ...            ...   
12076        2    332      451         1         1      1985              5   
12077        4   2844     2286         5         2      1980              5   
12078        2    472      600         1         1      1993              6   
12079        2    770      957         2         1      1930              1   
12080        2    124      139         4         1      1948              1   

       NUMFLRS  ROOFTYPE  NAPTFLRS  ...  TOTALBTURF

In [14]:
to_normalize = high_cor_df[high_cor_df.columns.difference(['KWH'])]

# minmax = MinMaxScaler((0,1)).fit(to_normalize)
# df_to_normalize = minmax.transform(df_to_normalize)

df_to_normalize = StandardScaler().fit_transform(to_normalize)
df_to_normalize = pd.DataFrame(df_to_normalize)


result1_df = pd.concat([df_to_normalize, high_cor_df['KWH']],axis = 1)

result1_df.to_csv("./Data/high_cor_df.csv", index=False)